In [ ]:
!pip install langchain
!pip install chroma-hnswlib
!pip install lark
!pip install HuggingFace
!pip install panel
!pip install -U langchain-community
!pip install python-dotenv
!pip install pypdf
!pip install sentence_transformers
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import sys
import chromadb
sys.path.append('../..')

import panel as pn  # GUI
pn.extension()

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.embeddings import HuggingFaceBgeEmbeddings


In [ ]:
persist_directory = '/content/chroma'

In [ ]:
!rm -rf ./content/chroma # remove old database files if any

In [ ]:
loader = PyPDFLoader("/content/paper.pdf")
pages = loader.load()

In [ ]:
full_text = " ".join([page.page_content for page in pages])

# Initialize the RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

# Split the text into chunks
chunks = r_splitter.split_text(full_text)


In [ ]:
chunks[1] #just to check if it's working

'in both deep learning architectural and evaluation approaches, and dataset building for easy\nmethods comparison.\nKeywords Image forgery detection ·Image forensics ·Deep learning ·Copy-move ·\nSplicing ·DeepFake ·Survey\n/envelopebackMarcello Zanardelli\nm.zanardelli005@unibs.it\nFabrizio Guerrini\nfabrizio.guerrini@unibs.it\nRiccardo Leonardi\nriccardo.leonardi@unibs.it\nNicola Adami\nnicola.adami@unibs.it\n1Department of Information Engineering, CNIT – University of Brescia,\nVia Branze 38, 25134 Brescia, ItalyPublished online: 3 October 2022Multimedia Tools and Applications (2023) 82:17521–17566 1 Introduction\nThe worldwide spread of smart devices, which integrate increasing quality cameras and\nimage processing tools and “apps”, the ubiquity of desktop computers, and the fact that allthese devices are almost permanently connected with each other and to remotely locateddata servers through the Internet, have given ordinary people the possibility to collect, store,and process an e

In [ ]:
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
model.query_instruction = "为这个句子生成表示以用于检索相关文章："


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [57]:
# Assuming 'chunks' is a list of strings
documents = [Document(page_content=chunk) for chunk in chunks]

vectordb = Chroma.from_documents(
    documents=documents,  # Pass the list of Document objects
    embedding=model,
    persist_directory=persist_directory,
    client=chromadb.Client()
)

In [58]:
print(vectordb._collection.count())

103


In [62]:
query = "what are the datasets?"


In [65]:
docs = vectordb.max_marginal_relevance_search(query)

# print results
for doc in docs:
    print(doc.page_content)


This dataset is also used for pre-training the AlexNet model used by the authors. In thepaper, it is not clear which portion of the dataset is used for training and which for testing.Therefore, it is not possible to evaluate if and how much the reported results are affected bybias due to correlation between training and testing sets. In order to clear up these issues,the authors could have used different datasets for either phase instead, such as MICC-F2000or MICC-F600.17544 Multimedia Tools and Applications (2023) 82:17521–17566 3.3 Copy-moveandsplicingmethods
We now move on to discuss those methods designed to detect both copy-move and splicing
forgeries.
3.3.1 CozzolinoandVerdoliva [18]
In this work, the authors presented a deep learning approach that aims to extract a cameramodel noise pattern (referred to as “noise print”) as a means to detect forgeries.
A digital camera, due to the on-board processing operations carried out on the signal
aThe accuracy score is sub-par compared to